dataset information
|

In [ ]:
## simclue ##

from datasets import load_dataset
import jsonlines

###### ATTENTION: NEG and INSTRUCTION need edit!!!! #######

# QUERY_INSTRUCTION = "Represent the sentence for retrieving duplicate sentences;"
# DOC_INSTRUCTION = "Represent the sentence for retrieving duplicate sentences;"
QUERY_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："
DOC_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："
# read simclue
simclue_path = "../../dataset/simclue_public/train_rank.json"
simclue_output_path = "../../dataset/our-zh/simclue.jsonl"
simclue_data = []

with jsonlines.open(simclue_path) as reader:
    for obj in reader:
        simclue_data.append(obj)
    
keys_to_keep = ["query", "neg", "pos"]
keys_to_delete = [key for key in simclue_data[0].keys() if key not in keys_to_keep]

# add instruction
for item in simclue_data:
    item["pos"] = item["title"]
    item["neg"] = item["neg_title"]
    item["query"] = [QUERY_INSTRUCTION,item["query"]]
    item["neg"] = [DOC_INSTRUCTION,item["neg"]]
    item["pos"] = [DOC_INSTRUCTION,item["pos"]]
    for key in keys_to_delete:
        item.pop(key)
        
with jsonlines.open(simclue_output_path, mode='w') as writer:
    writer.write_all(simclue_data)

In [ ]:
!export TOKENIZERS_PARALLELISM=true

In [ ]:
from datasets import load_from_disk
from string import Template
from FlagEmbedding import FlagModel
import numpy as np
import jsonlines
from tqdm import tqdm
# from joblib import Parallel, delayed
import faiss
import os

np.random.seed(42)
###### ATTENTION: NEG and INSTRUCTION need edit!!!! #######

QUERY_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："
DOC_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："

QUERY_INSTRUCTION_FOR_EMBEDMODEL = "为这个句子生成表示以用于检索相关文章："

dataset_names = [
    "alpaca_gpt4",
    "belle_2m",
    "chatmed_consult",
    "csl",
    "dureader_robust",
    "firefly",
    "miracl",
    "mlqa",
    "webqa",
    "xlsum",
    "zhihu_kol"
]
dataset_paths_template = Template("../../dataset/m3e/$dataset_name.dataset")
dataset_output_paths_template = Template("../../dataset/our-zh/$dataset_name.jsonl")

model = FlagModel('BAAI/bge-base-zh-v1.5', 
                  query_instruction_for_retrieval="为这个句子生成表示以用于检索相关文章：",
                  use_fp16=True) 
keys_to_keep = ["query", "pos"]

dataset_spilt = {
    "csl":"csl",
}


for d in dataset_names:
    print("================{}==============".format(d))
    # if os.path.exists(dataset_output_paths_template.substitute(dataset_name=d)):
    #     continue
    dataset_path = dataset_paths_template.substitute(dataset_name=d)
    dataset = load_from_disk(dataset_path)
    dataset = dataset["train"]
    print("length:",len(dataset))
    # cut only 100
    # dataset = dataset[dataset_spilt.get(d,"train")]
    dataset = dataset.rename_columns({"text" : "query", "text_pos" : "pos"})
    # dataset = dataset.add_column("neg",[""]*len(dataset))
    keys_to_delete = [k for k in dataset.column_names if k not in keys_to_keep]
    dataset = dataset.remove_columns(keys_to_delete)
    
    # find the hard negative sample from the dataset
    queries = [x["query"] for x in dataset]#[:100]
    docs = [x["pos"] for x in dataset]#[:100]
    q_embeddings = model.encode_queries(queries)
    d_embeddings = model.encode(docs)
    # for each query, find the hard negative sample from 25-35
    # use numpy to find the hard negative sample
    # parrallel 
    negs = [None] * len(dataset)
    # def process_query(i):
    #     similarity_q = np.dot(q_embeddings[i], d_embeddings.T)
    #     negs[i] = docs[np.argsort(similarity_q)[::-1][np.random.randint(25, 36)]]
    #     # print(dataset[i])
    #     return
    
    # # for i in range(len(dataset)):
    # Parallel(n_jobs=-1,backend='threading')(delayed(process_query)(i) for i in tqdm(range(len(dataset))))
    # print(negs)
    
    # use Faiss gpu
    # Step 1: change data type
    
    q_embeddings = q_embeddings.astype("float32")
    d_embeddings = d_embeddings.astype("float32")
    
    # Step 2 : Instantiate a FAISS index
    dim = q_embeddings.shape[1]
    measure = faiss.METRIC_INNER_PRODUCT
    param = 'IVF{},PQ16'.format(len(dataset)//100)
    index = faiss.index_factory(dim, param, measure)
    index.train(d_embeddings)
    index.add(d_embeddings)
    print("index")
    
    # Step 5: Search the index
    index.nprobe = 10
    _,similarity_q_index = index.search(q_embeddings, 36)
    negs_index = np.random.randint(25, 36, size=(d_embeddings.shape[0],))
    negs = [docs[similarity_q_index[i][negs_index[i]]] for i in range(len(dataset))]
    # negs = [docs[negs_index[i]] for i in range(len(dataset))]
    # negs = [docs[np.argsort(similarity_q[i])[::-1][np.random.randint(25, 36)]] for i in range(len(dataset))]
    
    
    
    dataset = dataset.add_column("neg",negs)
    dataset = dataset.map(lambda x: {"query" : [QUERY_INSTRUCTION, x["query"]], "pos" : [DOC_INSTRUCTION, x["pos"]], "neg" : [DOC_INSTRUCTION, x["neg"]]})
    with jsonlines.open(dataset_output_paths_template.substitute(dataset_name=d), mode="w") as f:
        for x in dataset:#[:100]:
            f.write(x)

In [ ]:
from datasets import load_dataset
import jsonlines
output_file = "../../dataset/our-zh/marco_chinese.jsonl"
dataset = load_dataset('unicamp-dl/mmarco', 'chinese',trust_remote_code=True)
dataset = dataset['train']
QUERY_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："
DOC_INSTRUCTION = "为这个句子生成表示以用于检索相关文章："
dataset = dataset.rename_columns({"query" : "query", "pos" : "positive","neg":"negative"})
dataset = dataset.map(lambda x: {"query" : [QUERY_INSTRUCTION, x["query"]], "pos" : [DOC_INSTRUCTION, x["positive"]], "neg" : [DOC_INSTRUCTION, x["negative"]]})
with jsonlines.open(output_file, mode='w') as writer:
    for data in dataset:
        writer.write(data)

In [1]:
import os
import jsonlines
datasets_path = "../../dataset/our-zh/hard_negs/clean"
# for all the files
files = os.listdir(datasets_path)
files = [file for file in files if file.endswith(".jsonl")]

# in_domain_datasets = [
#     'cMedQA2',
#     'dureader_retrieval',
#     'marco_chinese',
#     'mcpr-ecom',
#     'mcpr-medical',
#     'mcpr-video',
#     'NCPPolicies',
#     't2ranking'
#     ]
# # # files
# files = ['clean'+file+'.jsonl' for file in in_domain_datasets]
data = []

for file_index,file in enumerate(files):
    if file.startswith("train"):
        continue
    if file.startswith("shenqing"):
        continue
    if file.startswith("zhihu"):
        continue
    line_count = sum(1 for line in open(os.path.join(datasets_path, file), 'r'))
    print(file[:-6],line_count)
    with jsonlines.open(os.path.join(datasets_path, file)) as reader:
        i = 0
        for obj in reader:
            obj["task_id"] = file_index
            if len(obj["neg"]) < 6:
                print(len(obj["neg"]))
            # obj["neg"] = obj["neg"][:6]
            # print(len(obj["neg"][:6])) n
            data.append(obj)
            i += 1
            # if i > 500000:
            #     break

output_path = "../../dataset/our-zh/train.jsonl"
with jsonlines.open(output_path, mode="w") as writer:
    writer.write_all(data)

10w_why 161023
NCPPolicies 3366
alpaca_gpt4 43894
cMedQA2 99483
chatmed_consult 252536
cmrc2018 9157
csl 379166
csquad 64215
dureader_retrieval 79158
dureader_robust 14088
economic_information_daily_clean 33046
marco_chinese 375525
mcpr-ecom 84337
mcpr-medical 45799
mcpr-video 73834
mlqa 52290
retrieval_data_llm 174659
simclue 371088
t2ranking 140566
webqa 34375
whys 33298
xlsum 77328


In [ ]:
import jsonlines
datasets_path = "../../dataset/our-zh/hard_negs/retrieval_data_llm.jsonl"
with jsonlines.open(datasets_path ) as reader:
    dataset = list(reader)
    
for obj in dataset:
    obj["neg"] = obj["neg"][:11]
    
with jsonlines.open(datasets_path , mode="w") as writer:
    writer.write_all(dataset)

In [1]:
import os
import jsonlines
datasets_path = "/data/WorkSpace/openbmb/dataset/medi2-data-jsonl/hard_negs/clean"
# for all the files
files = os.listdir(datasets_path)
files = [file for file in files if file.endswith(".jsonl")]

# in_domain_datasets = [
#     'cMedQA2',
#     'dureader_retrieval',
#     'marco_chinese',
#     'mcpr-ecom',
#     'mcpr-medical',
#     'mcpr-video',
#     'NCPPolicies',
#     't2ranking'
#     ]
# # # files
# files = ['clean'+file+'.jsonl' for file in in_domain_datasets]
data = []

for file_index,file in enumerate(files):
    if file.startswith("train"):
        continue
    if file.startswith("shenqing"):
        continue
    if file.startswith("zhihu"):
        continue
    line_count = sum(1 for line in open(os.path.join(datasets_path, file), 'r'))
    print(file[:-6],line_count)
    with jsonlines.open(os.path.join(datasets_path, file)) as reader:
        i = 0
        for obj in reader:
            # obj["task_id"] = file_index
            if len(obj["neg"]) < 6:
                print(len(obj["neg"]))
            # obj["neg"] = obj["neg"][:6]
            # print(len(obj["neg"][:6])) n
            data.append(obj)
            i += 1
            # if i > 500000:
            #     break

output_path = "/data/WorkSpace/openbmb/dataset/medi2-data-jsonl/hard_negs/clean.jsonl"
with jsonlines.open(output_path, mode="w") as writer:
    writer.write_all(data)

10000 0
10001 0
10002 0
10003 2347
10004 0
10005 0
10006 0
10007 0
10008 0
10009 0
10010 143
10011 0
10012 349
10013 0
10014 4
10015 6426
10016 10
10017 165
10018 0
10019 0
10020 21
10021 151
10022 3
10023 682
10024 127
10025 0
10026 1343
10027 128646
10028 0
10029 0
10030 0
10031 0
10032 5655
10033 1784
10034 4426
10035 234
10036 1795
10037 0
10038 57
10039 5496
10040 255
10041 0
10042 44789
10043 2054
10044 0
10045 0
10046 1874
10047 95
10048 0
10049 0
10050 592
10051 0
10052 51
10053 2
10054 0
10055 0
10056 473
10057 0
10058 408
10059 113
10060 389012
10061 0
10062 175409
10063 0
10064 0
10065 0
10066 0
10067 123
10068 170298
10069 0
10070 0
10071 64
10072 0
10073 0
10074 0
10075 0
10076 2853
10077 753
10078 0
10079 0
10080 0
10081 85
10082 98
10083 0
10084 0
10085 0
10086 2879
10087 291
10088 881
10089 0
10090 227
10091 0
10092 0
10093 129
10094 0
10095 334
10096 0
10097 0
10098 24
10099 0
10100 0
10101 0
10102 620
10103 380
10104 0
10105 0
10106 18
10107 0
10108 0
10109 3716
10110

In [2]:
import os
import jsonlines
datasets_path = "/data/WorkSpace/openbmb/dataset/medi2-data-jsonl/hard_negs/clean.jsonl"

# in_domain_datasets = [
#     'cMedQA2',
#     'dureader_retrieval',
#     'marco_chinese',
#     'mcpr-ecom',
#     'mcpr-medical',
#     'mcpr-video',
#     'NCPPolicies',
#     't2ranking'
#     ]
data = []
with jsonlines.open(datasets_path ) as reader:
    for obj in reader:
        # obj["task_id"] = file_index
        if len(obj["neg"]) < 6:
            print(len(obj["neg"]))
        obj["neg"] = obj["neg"][:6]
        # print(len(obj["neg"][:6])) n
        data.append(obj)
        i += 1
        # if i > 500000:
        #     break

output_path = "/data/WorkSpace/openbmb/dataset/medi2-data-jsonl/hard_negs/clean_5negs.jsonl"
with jsonlines.open(output_path, mode="w") as writer:
    writer.write_all(data)